<a href="https://colab.research.google.com/github/mkrsteska/BSA2020_Team_Tissot_Project_2/blob/master/code/Classification_with_Universal_Sentence_Encoder_and_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
from tqdm import tqdm
import numpy as np
import pandas as pd

In [0]:
use = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

In [0]:
sent_1 = ["the location is great"]
sent_2 = ["amazing location"]

In [0]:
emb_1 = use(sent_1)
emb_2 = use(sent_2)

In [5]:
print(emb_1.shape)

(1, 512)


In [0]:
df = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

df = df[["text", "target"]]

In [8]:
df.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [0]:
train_text = df.text
test_text = df_test.text

y_train = df.target

In [10]:
X_train = []
for r in tqdm(train_text):
  emb = use([r])
  text_emb = tf.reshape(emb, [-1]).numpy()
  X_train.append(text_emb)
X_train = np.array(X_train)

100%|██████████| 7613/7613 [07:06<00:00, 17.87it/s]


In [11]:
X_test = []
for r in tqdm(test_text):
  emb = use([r])
  text_emb = tf.reshape(emb, [-1]).numpy()
  X_test.append(text_emb)

X_test = np.array(X_test)

100%|██████████| 3263/3263 [03:12<00:00, 16.99it/s]


In [12]:
print(X_train.shape, X_test.shape)

(7613, 512) (3263, 512)


In [13]:
print(X_train.shape, y_train.shape)

(7613, 512) (7613,)


In [0]:
model = keras.Sequential()

model.add(keras.layers.Dense(units=256, input_shape=(X_train.shape[1], ),activation='relu'))

model.add(keras.layers.Dropout(rate=0.5))

model.add(keras.layers.Dense(units=128, activation='relu'))

model.add(keras.layers.Dropout(rate=0.5))

model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy', 
    optimizer=keras.optimizers.Adam(0.001),
    metrics=['accuracy']
)

In [15]:
history = model.fit(
    X_train, y_train, 
    epochs = 10, 
    batch_size = 512,
    validation_split = 0.1,
    verbose = 1,
    shuffle = True
)

Epoch 1/10
14/14 [==============================] - 0s 26ms/step - loss: 0.6496 - accuracy: 0.6817 - val_loss: 0.5584 - val_accuracy: 0.8110
Epoch 2/10
14/14 [==============================] - 0s 15ms/step - loss: 0.5017 - accuracy: 0.8024 - val_loss: 0.3952 - val_accuracy: 0.8333
Epoch 3/10
14/14 [==============================] - 0s 16ms/step - loss: 0.4414 - accuracy: 0.8091 - val_loss: 0.3686 - val_accuracy: 0.8346
Epoch 4/10
14/14 [==============================] - 0s 15ms/step - loss: 0.4105 - accuracy: 0.8251 - val_loss: 0.3583 - val_accuracy: 0.8412
Epoch 5/10
14/14 [==============================] - 0s 15ms/step - loss: 0.3935 - accuracy: 0.8354 - val_loss: 0.3544 - val_accuracy: 0.8491
Epoch 6/10
14/14 [==============================] - 0s 15ms/step - loss: 0.3820 - accuracy: 0.8391 - val_loss: 0.3542 - val_accuracy: 0.8465
Epoch 7/10
14/14 [==============================] - 0s 15ms/step - loss: 0.3702 - accuracy: 0.8459 - val_loss: 0.3527 - val_accuracy: 0.8504
Epoch 8/10
14

In [0]:
df_test = pd.read_csv("test.csv")
ids = df_test['id'].to_numpy()

In [17]:
test_predict = model.predict_classes(X_test)[:, 0]

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [0]:
pd.DataFrame({'id': ids, 'target': test_predict}).to_csv('5. Submission_Tensorflow_Keras.csv', index=False)

In [0]:
#????????????????????????
model = keras.Sequential()

model.add(keras.layers.Conv1D(32, kernel_size=3, activation='relu', padding='same',input_shape=(512, 1)))

model.add(keras.layers.Conv1D(32, kernel_size=3, activation='relu', padding='same'))
model.add(keras.layers.Conv1D(32, kernel_size=3, activation='relu', padding='same'))
model.add(keras.layers.MaxPooling1D(pool_size=3))

model.add(keras.layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))

model.add(keras.layers.Dense(512, activation='sigmoid'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(512, activation='sigmoid'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Dense(512, activation='sigmoid'))
model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(0.001), metrics=['accuracy'])


In [122]:
history = model.fit(
    X_train, y_train, 
    epochs = 10, 
    batch_size = 512,
    validation_split = 0.1,
    verbose = 1,
    shuffle = True
)

Epoch 1/10


ValueError: ignored